![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [3]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [4]:
display(ds_jobs_transformed.describe(exclude='number'))

,city,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,19158,14650,19158,18772,18698,16345,19093,13220,13018,18735
unique,123,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,4355,13221,13792,13817,11598,14492,3286,3083,9817,8040


Columns containing categories with only two factors must be stored as Booleans (bool)

In [5]:
ds_jobs_transformed["relevant_experience"] = ds_jobs_transformed["relevant_experience"].astype(bool)

ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].astype(bool)
print(ds_jobs_transformed[["relevant_experience", "job_change"]].dtypes)

relevant_experience    bool
job_change             bool
dtype: object


Columns containing integers only must be stored as 32-bit integers (int32)

In [6]:
ds_jobs_transformed["student_id"] = ds_jobs_transformed["student_id"].astype("int32")
ds_jobs_transformed["training_hours"] = ds_jobs_transformed["training_hours"].astype("int32")

print(ds_jobs_transformed[["student_id","training_hours"]].dtypes)

student_id        int32
training_hours    int32
dtype: object


Columns containing floats must be stored as 16-bit floats (float16)

In [7]:
ds_jobs_transformed["city_development_index"] = ds_jobs_transformed["city_development_index"].astype("float16")

print(ds_jobs_transformed["city_development_index"].dtypes)

float16


Columns containing nominal categorical data must be stored as the category data type

In [8]:
ds_jobs_transformed["city"] = ds_jobs_transformed["city"].astype("category")
ds_jobs_transformed["gender"] = ds_jobs_transformed["gender"].astype("category")
ds_jobs_transformed["major_discipline"] = ds_jobs_transformed["major_discipline"].astype("category")
ds_jobs_transformed["company_type"] = ds_jobs_transformed["company_type"].astype("category")

print(ds_jobs_transformed[["city", "gender", "major_discipline", "company_type"]].dtypes)

city                category
gender              category
major_discipline    category
company_type        category
dtype: object


Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column

In [9]:
# Check the unique values of 'object' data types 
object_cols = ['enrolled_university', 'education_level',  "experience",
               'company_size', 'last_new_job']

# print unique values
for col in object_cols:
    print(col, ': ', [*ds_jobs_transformed[col].unique()])

enrolled_university :  ['no_enrollment', 'Full time course', nan, 'Part time course']
education_level :  ['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School']
experience :  ['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4', '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan]
company_size :  [nan, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10-49', '100-499', '500-999']
last_new_job :  ['1', '>4', 'never', '4', '3', '2', nan]


In [10]:
ordered_categories = {"enrolled_university": ['no_enrollment', 'Part time course', 'Full time course'],
                     "education_level": ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
                     "experience": ['<1'] + list(map(str, range(1, 21))) + ['>20'],
                     "company_size": ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
                     "last_new_job": ['never', '1', '2', '3', '4', '>4']}
print(ordered_categories)

{'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'], 'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], 'experience': ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'], 'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], 'last_new_job': ['never', '1', '2', '3', '4', '>4']}


In [11]:
def convert_type(col, type_='', order=[]):
    if len(order) < 1:
    	return ds_jobs_transformed[col].astype(type_)
    else: 
        return pd.Categorical(ds_jobs_transformed[col], ordered=True, categories=order)

In [12]:
for col in ordered_categories.keys():
    ds_jobs_transformed[col] = convert_type(col, order=ordered_categories[col])

The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies

In [13]:
# Filter the DataFrame based on the numeric values
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed["experience"] >= "10") &  
                                          (ds_jobs_transformed["company_size"] >= "1000-4999")]

In [14]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)

## Conclusions

To check and compare memory usage of the two dataframes

In [15]:
# Dataframe 1

ds_jobs_memory_usage = ds_jobs.memory_usage(deep = True).sum()

print(f'Total memory usage of the ds_jobs dataframe is {ds_jobs_memory_usage} bytes')

Total memory usage of the ds_jobs dataframe is 12391705 bytes


In [16]:
# Dataframe 2 

ds_jobs_clean_memory_usage = ds_jobs_transformed.memory_usage(deep = True).sum()

print(f'Total memory usage of the ds_jobs_clean dataframe after optimization is {ds_jobs_clean_memory_usage} bytes')

Total memory usage of the ds_jobs_clean dataframe after optimization is 81348 bytes


In [18]:
# Percentage decrease in memory usage

diff2 = ds_jobs_memory_usage - ds_jobs_clean_memory_usage
percent_decrease2 = (diff2/ds_jobs_memory_usage) *100

print(f'Upon filtering, we obtained a {percent_decrease2:.2f}% decrease in memory usage.')

Upon filtering, we obtained a 99.34% decrease in memory usage.


The difference in memory usage estimates is because .info() provides an approximate value, while .memory_usage(deep=True).sum() offers a precise calculation by fully accounting for object and category columns.

Reducing the memory usage of this DataFrame involved optimizing the data types used to store its columns.

-     By default, Pandas stores integer columns as 64-bit integers (int64) and float columns as 64-bit floats (float64), which consume more memory than necessary, especially for large datasets.By converting these columns to int32 and float16 respectively, I reduced the memory footprint of these columns while still preserving the data integrity within the precision limits of these data types.

-     Nominal categories are stored as object data type by default. This again, consumes significant memory, especially since there were limited number of unique categories repeated across many rows. Converting these nominal categorical columns to the categorical data type helps Pandas to internally represent each unique category with an interger thereby reducing memory usage.

-     When ordinal data is stored as regular categorical data - object - it doesn't preserve the order between categories. Changing the data type makes pandas maintain the order between categories to ensure that operations like sorting, comparison and even plotting of categorical plots are done correctly.
